`comparaison_env`

In [1]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *

Variables globales

In [2]:
### MODELNET40 REMESHING ISO
ModelNet40_aligned_us = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R22.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print("Fichiers bvs de Modelnet40 :", len(paths_bvs))
dir_bvs = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/bvs_remeshing_iso"

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
dir_us = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude.csv")
label_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_rep_etude_label.csv")
data_us_cam['label'] = list(label_us_cam['label'])
## Choix et bvs des 44 modeles 
dir_bvs_us = '/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific/csv_etude_filtre/visualisation_filtre'
paths_bvs_us_csv = glob.glob(os.path.join(dir_bvs_us, "**", "*global_distribution_label.csv*"), recursive=True); print("Modeles de US : ",len(paths_bvs_us_csv))
# Path data folder of user study
dir_Data = "/home/pelissier/These-ATER/Papier_internationale2/Data"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'aeroplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet', 'night_stand': 'cabinet', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}
# Outputs tmp
# Path of user-study outputs folder in Dataset
dir_outputs = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/user_study"

Fichiers bvs de Modelnet40 : 844
Modeles de US :  44


In [3]:
data_us_cam

,Unnamed: 0,R,i,j,theta,delta,X_rep_etude,Y_rep_etude,Z_rep_etude,label
0,0,2.2,0.0,0.0,1.570796,0.000000,1.347111e-16,2.200000,0.000000e+00,dessus
1,1,2.2,1.0,0.0,1.570796,0.785398,9.525517e-17,2.200000,9.525517e-17,dessus
2,2,2.2,2.0,0.0,1.570796,1.570796,8.248679e-33,2.200000,1.347111e-16,dessus
3,3,2.2,3.0,0.0,1.570796,2.356194,-9.525517e-17,2.200000,9.525517e-17,dessus
4,4,2.2,4.0,0.0,1.570796,3.141593,-1.347111e-16,2.200000,1.649736e-32,dessus
5,5,2.2,5.0,0.0,1.570796,3.926991,-9.525517e-17,2.200000,-9.525517e-17,dessus
6,6,2.2,6.0,0.0,1.570796,4.712389,-2.474604e-32,2.200000,-1.347111e-16,dessus
7,7,2.2,7.0,0.0,1.570796,5.497787,9.525517e-17,2.200000,-9.525517e-17,dessus
8,8,2.2,0.0,1.0,0.785398,0.000000,1.555635e+00,1.555635,0.000000e+00,dessus_face
9,9,2.2,1.0,1.0,0.785398,0.785398,1.100000e+00,1.555635,1.100000e+00,dessus_face_droit


## Contraintes : quelles caméras de lUS on considère ?

In [26]:
## Condition
## Toutes les caméras 
#j_cam_us_ok = [0,1,2,3,4]

## Que les cameras de la couronne j==1
j_cam_us_ok = [1]

## Que les cameras de la couronne j==1 ou j == 2
#j_cam_us_ok = [1,2]

## Cameras

In [27]:
## User study
# Coordonnées des caméras de l'étude utilateur
X_us = []; Y_us = []; Z_us = []; labels_us = []
for j in j_cam_us_ok:
    X_us += list(data_us_cam.loc[data_us_cam['j'] == j]['X_rep_etude'])
    Y_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Y_rep_etude'])
    Z_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Z_rep_etude'])
    labels_us += list(data_us_cam.loc[data_us_cam['j'] == j]['label']) 
cams_us = np.around(np.column_stack((X_us, Y_us, Z_us, np.array([1]*len(X_us)))),3)

## ModeleNet 
# Coordonnées des 12 caméras de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])
cams_modelnet = np.column_stack((X_modelnet, Y_modelnet, Z_modelnet, np.array([1]*12)))

## BVS user study --> Tout OU que couronne 'Milieu-Dessus' + ordre en fonction des choix

In [28]:
all_bvs_us = {}
# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_bvs_us_csv in paths_bvs_us_csv:
    name = os.path.basename(path_bvs_us_csv).split('_')[0]
    df = pd.read_csv(path_bvs_us_csv)
    filter_df = df[df['label'].isin(labels_us)]
    filter_sorted_df = filter_df.sort_values(by='poids', ascending=False)
    label_bvs_mesh = list(filter_sorted_df.loc[filter_sorted_df['poids'] ==  max(list(filter_sorted_df['poids']))]['label'])
    position_bvs_mesh = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_mesh]]
    cam_bvs = np.concatenate([np.array(X_us)[position_bvs_mesh], np.array(Y_us)[position_bvs_mesh], np.array(Z_us)[position_bvs_mesh]]).T  
    all_bvs_us[name] = {'df': filter_sorted_df, 'label_bvs': label_bvs_mesh, 'cam_bvs': cam_bvs}
     

## Pour 1 categorie

In [30]:
categorie_modelnet = 'car'
categorie_us = match_ModelNet2US[categorie_modelnet]; print(categorie_modelnet, categorie_us)
## Fichiers BVS de la categorie
paths_bvs = [paths_bvs[i] for i in range(len(paths_bvs)) if categorie_modelnet in paths_bvs[i]]; print("Fichiers bvs de la categorie dispo :", len(paths_bvs), paths_bvs[0])
random.shuffle(paths_bvs)

car carVasa
Fichiers bvs de la categorie dispo : 38 car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4


### Mise en correspondance cameras : Pour 1 objet 3D

In [47]:
## ModelNet Random
# Mesh random qui a ete aligne avec le mesh de l'étude utilisateur
num = 0
for num in tqdm.tqdm(range(len(paths_bvs)*0+1)):
    path_mesh_bvs = paths_bvs[num]; print(path_mesh_bvs)
    name_modelnet =  '_'.join(os.path.basename(path_mesh_bvs).split('_')[:2])
    path_mesh_modelnet = os.path.join(ModelNet40_aligned_us, path_mesh_bvs+"_aligned_ok_US.obj"); #print(path_mesh_modelnet)
    mesh_modelnet = trimesh.load_mesh(path_mesh_modelnet)
    # Transformation de la mesh de ModelNet40 en mesh de l'étude utilisateur
    transformations = read_pkl(path_mesh_modelnet.replace(".obj", ".pkl"))
    # Transformation des 12 cameras
    cams_modelnet_mesh = cams_modelnet.copy()
    for n in range(0, len(transformations)):
        # Rotation R
        R = transformations[f"transformations{n}"]
        cams_modelnet_mesh = np.dot(R, cams_modelnet_mesh.T).T
        
    ## BVS ATTENTION DANS LE CALCULS DE LA BVS ON A BOUCLE SUR RANGE(1,13) ET NON PAS SUR RANGE(0,12)
    num_cam_bvs_modelnet = read_pkl(os.path.join(dir_bvs, os.path.basename(path_mesh_bvs)+"_bvs.pkl"))['bvs'].split('_')[-1]
    cam_bvs_modelnet = cams_modelnet_mesh[int(num_cam_bvs_modelnet)-1][:3]; print(f'Modelnet : cam_{num_cam_bvs_modelnet}', cam_bvs_modelnet)
    
    ###################################################################
    ## User stuy
    # mesh random from User_study
    path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj"); #print(path_mesh_us)
    mesh_us = trimesh.load_mesh(path_mesh_us)
    # BVS 
    cam_bvs_us = all_bvs_us[categorie_us]['cam_bvs']; print('US : ', cam_bvs_us, all_bvs_us[categorie_us]['label_bvs'])

    ## Superposition : US et modelent40
    #show_cams(mesh_modelnet, cams_modelnet_mesh, name_modelnet, mesh_us, cams_us, categorie_us,  dir_outputs)
    
    

100%|██████████| 1/1 [00:00<00:00, 10.53it/s]

car/train/car_0007_SMPLER_centered_scaled_remeshing_iso_iter8
Modelnet : cam_3 [ 1.11970189  1.02713081 -1.93423628]
US :  [[ 1.1         1.55563492 -1.1       ]] ['dessus_face_gauche']


# Debug

In [ ]:
# ## OBJ file with (12 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_modelnet = []#np.array(mesh_modelnet.vertices)
# faces_mesh_modelnet = []#np.array(mesh_modelnet.faces) 
# # colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
# colormap = plt.get_cmap('hot'); colors_modelnet = colormap(np.linspace(0, 1, 12))[::-1] 
# #with open(os.path.join(dir_outputs, categorie_modelnet+"_modelNet40+12cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_modelnet:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 12 cameras positions
#     vertex_offset = len(verts_mesh_modelnet)
#     for cube_center, cube_color in zip(cams_modelnet_mesh[:, :3], colors_modelnet[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_modelnet:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
        
# ################################################################################"" 
# ## OBJ file with (8 cameras + obj)
# # Vertices and faces of the model user study
# verts_mesh_us = np.array(mesh_us.vertices)
# faces_mesh_us = np.array(mesh_us.faces) 
# # colors : Jaune cam 1 --> Vert cam 3 ---> Bleu cam 5 --> Bleu foncé cam 8
# colormap = plt.get_cmap('hot'); colors_us = colormap(np.linspace(0, 1, 8))[::-1] 
# # Write obj
# #with open(os.path.join(dir_outputs, categorie_us+"_us+8cam.obj"), 'w') as obj_file:
#     # Write vertices
#     for vertex in verts_mesh_us:
#         obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
#     # Write 8 cameras positions
#     vertex_offset = len(verts_mesh_us)
#     for cube_center, cube_color in zip(cams_us[:, :3], colors_us[:,:3]):  
#         vertex_offset = add_cube_to_obj(obj_file, cube_center, cube_color, vertex_offset, cube_size=0.3)
#     # Write faces
#     for face in faces_mesh_us:
#         # Convert face indices to 1-based indexing
#         obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   

### BVS : match celle de l'US et celle de ModelNet40